## Exercise 1

Flux balance analysis is based on a very simple model of the reactions going on in a cell, keeping track only of the gross rate of consumption and production of various chemical species within the cell. Based on the known [stoichiometry](https://en.wikipedia.org/wiki/Stoichiometry) of the reactions, and known upper bounds on some of the reaction rates, we can compute bounds on the other reaction rates, or cell growth, for example.

We focus on $m$ metabolites in a cell, labeled $M_1$, ..., $M_m$. There are $n$ reactions going on, labeled $R_1$, ..., $R_n$, with nonnegative reaction rates $v_1$, ..., $v_n$. Each reaction has a (known) stoichiometry, which tells us the rate of consumption and production of the metabolites per unit of reaction rate. The stoichiometry data is given by the stoichiometry matrix $S \in \mathbb{R}^{m\times n}$, defined as follows: $S_{ij}$ is the rate of production of $M_i$ due to unit reaction rate $v_j = 1$. Here we consider consumption of a metabolite as negative production; so for example $S_{ij}=-2$ means that reaction $R_j$ causes metabolite $M_i$ to be consumed at a rate $2v_j$.

As an example, suppose reaction $R_1$ has the form $M_1 \to M_2 + 2M_3$. The consumption rate of $M_1$, due to this reaction, is $v_1$; the production rate of $M_2$ is $v_1$; and the production rate of $M_3$ is $2v_1$. The reaction $R_1$ has no effect on metabolites $M_4$, ..., $M_m$. This corresponds to a first column of $S$ of the form $(-1,1,2,0,\dots,0)$.

Reactions are also used to model flow of metabolites into and out of the cell. For example, suppose that reaction $R_2$ corresponds to the flow of metabolite $M_1$ into the cell, with $v_2$ giving the flow rate. This corresponds to a second column of $S$ of the form $(1,0,\dots,0)$.

The last reaction, $R_n$, corresponds to biomass creation, or cell growth, so the reaction rate $v_n$ is the cell growth rate. The last column of $S$ gives the amounts of metabolites used or created per unit of cell growth rate.

Since our reactions include metabolites entering or leaving the cell, as well as those converted to biomass within the cell, we have conservation of the metabolites, which can be expressed as $Sv=0$. In addition, we are given upper limits on some of the reaction rates, which we express as $v \preccurlyeq v^{\max}$. The goal is to find the maximum possible cell growth rate consistent with the constraints.



Consider the data

In [ ]:
import cvxpy as cp
import numpy as np
S = np.array([[1,	0,	0,	0,	0,	0],
              [-1,	1,	0,	0,	0,	0],
              [-1,	0,	1,	0,	0,	0],
              [0,	-1,	0,	2,	-1,	0],
              [0,	0,	0,	0,	1,	0],
              [0,	-2,	1,	0,	0,	1],
              [0,	0,	-1,	1,	0,	0],
              [0,	0,	0,	0,	0,	-1],
              [0,	0,	0,	-1,	0,	0]]).T

vmax = np.array([10.10,100,5.90,100,3.70,100,100,100,100])

*    What is the maximum possible cell growth rate $G^*$?
*    Which of the reaction rate limits have nonzero optimal Lagrange multipliers? You can find how to obtain the dual variables [here](https://www.cvxpy.org/tutorial/advanced/index.html#dual-variables).
*    You will see in the next lecture that the nonzero optimal Lagrange multipliers indicate the constraints towards which the optimal solution is sensitive. The larger the value of the optimal Lagrange multiplier, the larger is the deterioration of the optimal value if the corresponding constraint is made more strict. Which of the reaction rate limits is the maximum growth rate most sensitive to?

### Solution Exercise 1

In [ ]:
# We declare the variable, the objective and constraints
m = S.shape[0]
n = S.shape[1]

v = cp.Variable(n)
constraints = [S@v == 0, v >= 0, v <= vmax ]
obj = v[n-1]

In [ ]:
# We solve the problem
prob = cp.Problem(cp.Maximize(obj),constraints)
prob.solve()

print("The maximum possible cell growth rate is ", prob.value)

The maximum possible cell growth rate is  13.54999999695169


In [ ]:
# and find the optimal dual variables
print("Dual variables:",np.round(constraints[2].dual_value,2))

Dual variables: [0.5 0.  0.5 0.  1.5 0.  0.  0.  0. ]


Only three reactions are limited, namely $R_1$, $R_3$ and $R_5$. All other Lagrange multipliers are zero. The non-zero optimal
values are $0.5$, $0.5$, and $1.5$, respectively. So it appears that the limit on reaction $R_5$ is the one that would have the largest effect on the optimal growth rate, for a small change
in the limit.

# Exercise 2

This exercise is a continuation of the previous.

For simplicity, we'll assume that each reaction is controlled by an associated gene, i.e., gene $G_i$ controls reaction $R_i$. Knocking out a set of genes associated with some reactions has the effect of setting the reaction rates (or equivalently, the associated $v^{\max}$ entries) to zero, which of course reduces the maximum possible growth rate. If the maximum growth rate becomes small enough or zero, it is reasonable to guess that knocking out the set of genes will kill the cell. An *essential gene* is one that when knocked out reduces the maximum growth rate below a given threshold $G^{\min}$. (Note that $G_n$ is always an essential gene.) A *synthetic lethal* is a pair of non-essential genes that when knocked out reduces the maximum growth rate below the threshold. 

*    Given the threshold $G^{\min} = 0.2G^*$, find all essential genes.
*    Given the threshold $G^{\min} = 0.2G^*$, find all the gene pairs which are synthetic lethals.

### Solution Exercise 2

In [ ]:
Gmin = 0.2 * prob.value

In [ ]:
# We solve the same problem but we knock out one gene at the time
for i in range(n):
  vmax = np.array([10.10,100,5.90,100,3.70,100,100,100,100])
  vmax[i]=0
  constraints = [S@v == 0, v >= 0, v <= vmax ]
  obj = v[n-1]
  prob = cp.Problem(cp.Maximize(obj),constraints)
  prob.solve()
  # We check if the gene is essential
  if prob.value < Gmin:
    print("The gene ", i+1, " is essential. Gmax is ", prob.value)

The gene  1  is essential. Gmax is  2.714666078375953e-11
The gene  9  is essential. Gmax is  2.994131326743363e-11


In [ ]:
# Similar approach, but we check all pairs
for i in range(1,8):
  for j in range(1,8):
    if i<j:
      vmax = np.array([10.10,100,5.90,100,3.70,100,100,100,100])
      vmax[i]=0
      vmax[j]=0
      constraints = [S@v == 0, v >= 0, v <= vmax ]
      obj = v[n-1]
      prob = cp.Problem(cp.Maximize(obj),constraints)
      prob.solve()
      if prob.value < Gmin:
        print("The pair ", i+1," ",j+1, " is synthetic letal. Gmax is ", prob.value)

The pair  2   3  is synthetic letal. Gmax is  4.907085642708621e-12
The pair  2   7  is synthetic letal. Gmax is  1.7217180784949536e-10
The pair  4   7  is synthetic letal. Gmax is  5.269891693136436e-11
The pair  5   7  is synthetic letal. Gmax is  1.0770330670932224e-11
